In [1]:
!apt-get update 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz 
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [49.8 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

spark

In [3]:
!wget https://raw.githubusercontent.com/VVYing/StockerBot/master/Bitcoin_tweets.csv

--2021-03-31 19:07:15--  https://raw.githubusercontent.com/VVYing/StockerBot/master/Bitcoin_tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17870211 (17M) [text/plain]
Saving to: ‘Bitcoin_tweets.csv’

Bitcoin_tweets.csv  100%[===================>]  17.04M  --.-KB/s    in 0.1s    

2021-03-31 19:07:15 (163 MB/s) - ‘Bitcoin_tweets.csv’ saved [17870211/17870211]



Original data source on Kaggle：[Bitcoin Tweets](https://www.kaggle.com/kaushiksuresh147/bitcoin-tweets)


In [4]:
bt = spark.read.csv('Bitcoin_tweets.csv',header=True)
bt.show(5)

+--------------------+---------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+
|           user_name|  user_location|    user_description|       user_created|user_followers|user_friends|user_favourites|user_verified|               date|                text|            hashtags|              source|         is_retweet|
+--------------------+---------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+
|       DeSota Wilson|    Atlanta, GA|Biz Consultant, r...|2009-04-26 20:05:09|          8534|        7605|           4838|        False|2021-02-10 23:59:04|Blue Ridge Bank s...|         ['bitcoin']|     Twitter Web App|              False|
|            CryptoND|           nul

In [5]:
bt.dtypes

[('user_name', 'string'),
 ('user_location', 'string'),
 ('user_description', 'string'),
 ('user_created', 'string'),
 ('user_followers', 'string'),
 ('user_friends', 'string'),
 ('user_favourites', 'string'),
 ('user_verified', 'string'),
 ('date', 'string'),
 ('text', 'string'),
 ('hashtags', 'string'),
 ('source', 'string'),
 ('is_retweet', 'string')]

In [6]:
bt.describe().show()

+-------+--------------------+-------------------+--------------------+-----------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|summary|           user_name|      user_location|    user_description|     user_created|      user_followers|        user_friends|   user_favourites|       user_verified|                date|                text|            hashtags|              source|       is_retweet|
+-------+--------------------+-------------------+--------------------+-----------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+
|  count|              132005|              70942|               86713|            77543|               50229|               48919|             48640|               48597|       

In [6]:
from pyspark.sql.functions import col, split

In [7]:
bt = bt.withColumn("city", split(col("user_location"), ",").getItem(0)).withColumn("state", split(col("user_location"), ",").getItem(1))
bt.show(5)

+--------------------+---------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------+--------+
|           user_name|  user_location|    user_description|       user_created|user_followers|user_friends|user_favourites|user_verified|               date|                text|            hashtags|              source|         is_retweet|   city|   state|
+--------------------+---------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------+--------+
|       DeSota Wilson|    Atlanta, GA|Biz Consultant, r...|2009-04-26 20:05:09|          8534|        7605|           4838|        False|2021-02-10 23:59:04|Blue Ridge Bank s...|         ['bitcoin']|     Twitter Web App|      

In [8]:
ts = bt.select("text", "state")

In [9]:
ts = ts.dropDuplicates()
ts = ts.na.drop()
ts.count()

7140

In [10]:
text = bt.select("text", "is_retweet")

In [11]:
text = text.dropDuplicates()
text = text.na.drop()
text.count()

16033

### Abridge text data

In [12]:
from pyspark.ml import Pipeline 
from pyspark.sql.functions import udf,regexp_replace,isnull
from pyspark.sql.types import StringType,IntegerType

In [13]:
text = text.withColumn("only_str",regexp_replace(col('text'), '\d+', ''))
text.show(5)

+--------------------+-------------------+--------------------+
|                text|         is_retweet|            only_str|
+--------------------+-------------------+--------------------+
|Bitcoin Daily Par...|              False|Bitcoin Daily Par...|
|Get in the game #...|              False|Get in the game #...|
|"""We've done a l...|Twitter for Android|"""We've done a l...|
|Determining wheth...|              False|Determining wheth...|
|So there is a lot...|              False|So there is a lot...|
+--------------------+-------------------+--------------------+
only showing top 5 rows



In [14]:
text.select("text","only_str").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                         |only_str                                                                                                                                    |
+---------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Bitcoin Daily Parabolas 2/10/2021 #bitcoin #CryptoCurrency #Parabolas #btcusd #btc #trading #blockchain #crypto… https://t.co/XzAYKAycwi     |Bi

### Separate sentences into words

In [15]:
from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover

In [17]:
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")
words = regex_tokenizer.transform(text)
words.show()

+--------------------+-------------------+--------------------+--------------------+
|                text|         is_retweet|            only_str|               words|
+--------------------+-------------------+--------------------+--------------------+
|Bitcoin Daily Par...|              False|Bitcoin Daily Par...|[bitcoin, daily, ...|
|Get in the game #...|              False|Get in the game #...|[get, in, the, ga...|
|"""We've done a l...|Twitter for Android|"""We've done a l...|[we, ve, done, a,...|
|Determining wheth...|              False|Determining wheth...|[determining, whe...|
|So there is a lot...|              False|So there is a lot...|[so, there, is, a...|
|EY, Quadriga Law ...|              False|EY, Quadriga Law ...|[ey, quadriga, la...|
|👍👍#ATOM COSMOS ...|              False|👍👍#ATOM COSMOS ...|[atom, cosmos, bi...|
|🤝 Follow me on @...|              False|🤝 Follow me on @...|[follow, me, on, ...|
|Checkout our COIN...|              False|Checkout our COIN...|[checkou

### Remove StopWords

In [18]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_rm = remover.transform(words)
words_rm.select("words","filtered").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|words                                                                                                                                                             |filtered                                                                                                                          |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|[bitcoin, daily, parabolas, bitcoin, cryptocurrency, parabolas, btcusd, btc, trading, blockchain, crypto, https

### Encode column of is_retweet to a column of is_retweet indices

In [19]:
indexer = StringIndexer(inputCol="is_retweet", outputCol="is_retweetIndex")
feature = indexer.fit(words_rm).transform(words_rm)
feature.show()

+--------------------+-------------------+--------------------+--------------------+--------------------+---------------+
|                text|         is_retweet|            only_str|               words|            filtered|is_retweetIndex|
+--------------------+-------------------+--------------------+--------------------+--------------------+---------------+
|Bitcoin Daily Par...|              False|Bitcoin Daily Par...|[bitcoin, daily, ...|[bitcoin, daily, ...|            0.0|
|Get in the game #...|              False|Get in the game #...|[get, in, the, ga...|[get, game, crypt...|            0.0|
|"""We've done a l...|Twitter for Android|"""We've done a l...|[we, ve, done, a,...|[ve, done, lot, t...|            3.0|
|Determining wheth...|              False|Determining wheth...|[determining, whe...|[determining, whe...|            0.0|
|So there is a lot...|              False|So there is a lot...|[so, there, is, a...|[lot, talk, elonm...|            0.0|
|EY, Quadriga Law ...|  

### Convert text into vectors of token counts

In [21]:
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(feature)
countVectorizer_feateures = model.transform(feature)

### Model training and evaluation

In [22]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [23]:
(trainingData, testData) = countVectorizer_feateures.randomSplit([0.8, 0.2],seed = 11)

In [24]:
nb = NaiveBayes(modelType="multinomial",labelCol="is_retweetIndex", featuresCol="features")
nbModel = nb.fit(trainingData)
nb_predictions = nbModel.transform(testData)

In [25]:
nb_predictions.select("prediction", "is_retweetIndex", "features").show(5)

+----------+---------------+--------------------+
|prediction|is_retweetIndex|            features|
+----------+---------------+--------------------+
|       0.0|            0.0|(29251,[0,1,3,44,...|
|       0.0|            0.0|(29251,[2,3,75,12...|
|       0.0|            0.0|(29251,[0,1,303,3...|
|       0.0|            0.0|(29251,[2,3,19,44...|
|       0.0|            0.0|(29251,[0,1,2,3,1...|
+----------+---------------+--------------------+
only showing top 5 rows



In [26]:
evaluator = MulticlassClassificationEvaluator(labelCol="is_retweetIndex", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))
print("Test Error of NaiveBayes = %g " % (1.0 - nb_accuracy))

Accuracy of NaiveBayes is = 0.991888
Test Error of NaiveBayes = 0.00811232 
